This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [92]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [93]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "/kaggle/input/pizza-classification-data/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "/kaggle/input/pizza-classification-data/test"    # e.g. './muffin-vs-chihuahua/test'

In [94]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [95]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 1280 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 366 images belonging to 2 classes.


In [96]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [97]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [98]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 34s 786ms/step - accuracy: 0.5138 - loss: 1.2773 - val_accuracy: 0.5938 - val_loss: 0.9220
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 692ms/step - accuracy: 0.5721 - loss: 0.9007 - val_accuracy: 0.5500 - val_loss: 0.8429
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 677ms/step - accuracy: 0.5950 - loss: 0.8291 - val_accuracy: 0.5813 - val_loss: 0.7876
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 681ms/step - accuracy: 0.6124 - loss: 0.7777 - val_accuracy: 0.5688 - val_loss: 0.7722
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 30s 742ms/step - accuracy: 0.6272 - loss: 0.7546 - val_accuracy: 0.6938 - val_loss: 0.7342
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 695ms/step - accuracy: 0.6184 - loss: 0.7392 - val_accuracy: 0.6094 - val_loss: 0.7450
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 30s 737ms/step - accuracy: 0.6498 - loss: 0.7155 - val_accuracy: 0.6750 - val_loss: 0.6936
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 677ms/step - accuracy: 0.6755 - loss: 0.6788 - val_accu

In [99]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 245ms/step - accuracy: 0.7337 - loss: 0.6838
Test Accuracy: 0.7650273442268372


In [100]:
# SAVE THE MODEL
model.save('/kaggle/working/models/exercise_6_custom_tambalo.h5')

In [101]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='/kaggle/working/models/exercise_6_custom_tambalo.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Pizza" if pred >= 0.5 else "Not Pizza"
    print(f"Prediction: {label} (confidence: {pred:.2f})")
    return pred


In [102]:
# Example usage:
pred1 = predict_image("/kaggle/input/pizza-classification-data/test/not_pizza/3140015.jpg")
pred2 = predict_image("/kaggle/input/pizza-classification-data/test/not_pizza/3194027.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


Prediction: Pizza (confidence: 0.70)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Prediction: Not Pizza (confidence: 0.38)
